# 304 - Primonacci

## Problem Statement

For any positive integer $n$ the function $\operatorname{next\_prime}(n)$ returns the smallest prime $p$ such that $p \gt n$.

The sequence $a(n)$ is defined by:<br>
$a(1)=\operatorname{next\_prime}(10^{14})$ and $a(n)=\operatorname{next\_prime}(a(n-1))$ for $n \gt 1$.

The Fibonacci sequence $f(n)$ is defined by:
$f(0)=0$, $f(1)=1$ and $f(n)=f(n-1)+f(n-2)$ for $n \gt 1$.

The sequence $b(n)$ is defined as $f(a(n))$.

Find $\sum b(n)$ for $1 \le n \le 100\,000$. 
Give your answer mod $1234567891011$. 

## Solution

There are two main challenges to this problem. The first one is to quickly compute the next prime. This is easily done using ```sympy```. The second challenge is to efficiently compute the $n$-th Fibonacci number, $F_n$. To do so, we can use the relationship

\begin{equation}
    \begin{bmatrix} 1 & 1 \\ 
    1 & 0 \end{bmatrix}^n = \begin{bmatrix} F_{n+1} & F_n \\ 
    F_n & F_{n-1} \end{bmatrix}.
\end{equation}

In order to do the exponentiation is a reasonable time, we use the exponentiation by squaring technique which is based on the observation that

\begin{equation}
    x^n = \begin{cases}
        x(x^2)^\frac{n - 1}{2}, & \text{if } n \text{ is odd} \\
        (x^2)^{\frac{n}{2}}, & \text{if } n \text{ is even}
        
    \end{cases}.
\end{equation}

This allows to perform matrix exponentiation in $O(\log n)$. Note the use of ```dtype = object``` instead of ```dtype = np.int64``` in the implementation to avoid overflow.

Therefore, to obtain the solution, we simply compute the next prime and add the corresponding Fibonacci number to a running sum modulo 1234567891011. 

Note that the solution below could be improved by keeping the matrix instead of fully redoing the exponentiation from zero. 

In [2]:
import numpy as np
from sympy import nextprime

def matrix_modular_exponentiation(matrix, power, mod):
    """ Matrix modular exponentiation by squaring. Set dtype to object to avoid overflow"""
    # Ensure the matrix is in integer format for modular operations
    matrix = np.array(matrix, dtype=object)
    # Start with the identity matrix of the same size as the input matrix
    result = np.eye(matrix.shape[0], dtype=object)

    # Apply exponentiation by squaring technique
    while power > 0:
        if power % 2 == 1:
            result = np.dot(result, matrix) % mod
        matrix = np.dot(matrix, matrix) % mod
        power //= 2

    return result


def fibonacci_mod(n, mod):
    """ Compute nth Fibonacci number using matrix exponentiation"""
    if n == 0:
        return 0
    elif n == 1:
        return 1
    else:
        F = [[1, 1], [1, 0]]
        result_matrix = matrix_modular_exponentiation(F, n - 1, mod)
        return result_matrix[0, 0]


res = 0
mod = 1234567891011
prime = 10**14
for i in range(100000):
    prime = nextprime(prime)
    res = (res + fibonacci_mod(prime, mod)) % mod

res

283988410192